In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display

import requests

In [18]:
offline = True
city="Koper"
#city="Ljubljana"

In [19]:
weather_groups = {'Clear': 'dry',
                  'Partially cloudy': 'dry',
                  'Rain, Partially cloudy': 'humid',
                  'Rain': 'wet',
                  'Rain, Overcast': 'wet',
                  'Overcast': 'humid'}

In [30]:
if not offline:
    year = "2021"
    
    url = "https://visual-crossing-weather.p.rapidapi.com/history"

    querystring = {"startDateTime":f"{year}-01-01T12:00:00","aggregateHours":"24","location":f"{city},Slovenia","endDateTime":f"{year}-12-31T20:00:00","unitGroup":"metric","contentType":"csv","shortColumnNames":"0"}
    #querystring = {"startDateTime":"2020-01-01T12:00:00","aggregateHours":"24","location":"Ljubljana,Slovenia","endDateTime":"2020-12-31T20:00:00","unitGroup":"metric","contentType":"csv","shortColumnNames":"0"}
    #querystring = {"startDateTime":"2019-01-01T12:00:00","aggregateHours":"24","location":"Ljubljana,Slovenia","endDateTime":"2019-12-31T20:00:00","unitGroup":"metric","contentType":"csv","shortColumnNames":"0"}

    headers = {
        'x-rapidapi-key': "replace with your API key",
        'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    if city=="Ljubljana":
        f = open(f'data\\weather_raw_{year}.csv', 'w')
    else:
        f = open(f'data\\weather_raw_{year}_{city}.csv', 'w')
    f.write(str(response.text))
    f.close()

    

In [21]:
if city=="Ljubljana":
    df = pd.read_csv('data\\weather_raw_2019.csv')
    df2 = pd.read_csv('data\\weather_raw_2020.csv')
    df = pd.concat((df, df2), ignore_index=True)
    df2 = pd.read_csv('data\\weather_raw_2021.csv')
    df = pd.concat((df, df2), ignore_index=True)
else:
    df = pd.read_csv(f'data\\weather_raw_2019_{city}.csv')
    df2 = pd.read_csv(f'data\\weather_raw_2020_{city}.csv')
    df = pd.concat((df, df2), ignore_index=True)
    df2 = pd.read_csv(f'data\\weather_raw_2021_{city}.csv')
    df = pd.concat((df, df2), ignore_index=True)

In [25]:
df=df[['Date time', 'Temperature', 'Conditions']].copy()
df['Date time'] = df['Date time'].map(lambda x: x.split('/')[2]+'-'+x.split('/')[0]+'-'+x.split('/')[1])

In [26]:
df['weather'] = df.apply(lambda x: 'good' if (weather_groups[x['Conditions']]=='dry' and x['Temperature'] >=5) or (weather_groups[x['Conditions']]=='humid' and x['Temperature'] >=10) else 'bad', axis=1)    

In [27]:
df[df['Date time'] == '2019-10-05']

,Date time,Temperature,Conditions,weather
277,2019-10-05,15.1,Clear,good


In [11]:
df = df[['Date time', 'weather']]
df.columns = ['date', 'weather']

In [28]:
if city == 'Ljubljana':
    df.to_csv('data\\weather.csv', index=False)
else:
    df.to_csv(f'data\\weather_{city}.csv', index=False)